In [1]:
from dotenv import load_dotenv
load_dotenv()

import os

from agential.core.llm import LLM

llm = LLM("gpt-3.5-turbo", organization=os.getenv("OPENAI_ORGANIZATION"))

In [6]:
from agential.agents.self_refine.prompts import GSM8K_CRITIQUE_FEWSHOT_EXAMPLES, GSM8K_REFINE_FEWSHOT_EXAMPLES, HUMANEVAL_CRITIQUE_FEWSHOT_EXAMPLES, HUMANEVAL_REFINE_FEWSHOT_EXAMPLES, SELF_REFINE_CRITIQUE_INSTRUCTION_GSM8K, SELF_REFINE_CRITIQUE_INSTRUCTION_HUMANEVAL, SELF_REFINE_INSTRUCTION_GSM8K, SELF_REFINE_INSTRUCTION_HUMANEVAL, SELF_REFINE_REFINE_INSTRUCTION_GSM8K, SELF_REFINE_REFINE_INSTRUCTION_HUMANEVAL
from agential.agents.self_refine.strategies.code import SelfRefineCodeStrategy
from agential.agents.self_refine.strategies.math import SelfRefineMathStrategy
from agential.core.fewshots.gsm8k import GSM8K_FEWSHOT_EXAMPLES_COT, GSM8K_FEWSHOT_EXAMPLES_POT
from agential.core.fewshots.humaneval import HUMANEVAL_FEWSHOT_EXAMPLES_POT

inst = {
    "task_id": "HumanEval/0",
    "prompt": 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n',
    "entry_point": "has_close_elements",
    "canonical_solution": "    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n",
    "test": "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False\n    assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False\n\n",
}
question = inst["prompt"]
tests = f"{inst['test']}\ncheck({inst['entry_point']})"

strategy = SelfRefineCodeStrategy(llm=llm, testing=True)

out = strategy.generate(
    question=question,
    examples=HUMANEVAL_FEWSHOT_EXAMPLES_POT,
    prompt=SELF_REFINE_INSTRUCTION_HUMANEVAL,
    critique_examples=HUMANEVAL_CRITIQUE_FEWSHOT_EXAMPLES,
    critique_prompt=SELF_REFINE_CRITIQUE_INSTRUCTION_HUMANEVAL,
    refine_examples=HUMANEVAL_REFINE_FEWSHOT_EXAMPLES,
    refine_prompt=SELF_REFINE_REFINE_INSTRUCTION_HUMANEVAL,
    additional_keys={},
    critique_additional_keys={"tests": tests},
    refine_additional_keys={"tests": tests},
    max_interactions=3,
    reset=True,
)

<PROMPT AGENT============================================================>
You are an AI that only responds with python code, NOT ENGLISH. You will be given a function signature and its docstring by the user. 

```python
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """

    pass
```
<PROMPT AGENT============================================================>
<OUT AGENT============================================================>
'```python\ndef has_close_elements(numbers, threshold):\n    return any(abs(a - b) < threshold for i, a in enumerate(numbers) for b in numbers[i + 1:])\n```'
<OUT AGENT============================================================>
<PROMPT CRITIque========================

In [7]:
print(out.answer)

```python
def has_close_elements(numbers, threshold):
    return any(abs(numbers[i] - numbers[j]) < threshold for i in range(len(numbers)) for j in range(i+1, len(numbers))
```


In [9]:
for i in out.additional_info:
    print(i.answer)
    print()
    print()

```python
def has_close_elements(numbers, threshold):
    return any(abs(a - b) < threshold for i, a in enumerate(numbers) for b in numbers[i + 1:])
```


```python
def has_close_elements(numbers, threshold):
    return any(abs(numbers[i] - numbers[j]) < threshold for i in range(len(numbers)) for j in range(i+1, len(numbers))


METADATA = {
    'author': 'jt',
    'dataset': 'test'
}
```


```python
def has_close_elements(numbers, threshold):
    return any(abs(numbers[i] - numbers[j]) < threshold for i in range(len(numbers)) for j in range(i+1, len(numbers))
```




: 